In [83]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
from tensorflow.keras.models import load_model 
import pickle 


In [84]:
## Load the trained model and scaler pickle file nd one hot encoding file 
model=load_model('model.h5')

# Load the encoders and scaler 
with open('one_hot_encoder_geo.pkl','rb')as file:
    one_hot_encoder=pickle.load(file)

with open('label_encoder_gender.pkl','rb')as file:
    label_encoder_gender=pickle.load(file)

with open('scaler.pkl','rb')as file:
    scaler=pickle.load(file)

In [85]:
## Example input data 
input_data={
    'CreditScore':600, 
    'Geography':'France',
    'Gender':'Male',
    'Age':20,
    'Tenure':3,
    'Balance':20000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMemeber':1,
    'EstimatedSalary':50000
}

In [86]:
## One Hot Encoding of Geography column 
geo_encoded=one_hot_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df= pd.DataFrame(geo_encoded,columns=one_hot_encoder.get_feature_names_out(['Geography']))

c:\Users\vpoor\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [87]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [88]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMemeber,EstimatedSalary
0,600,France,Male,20,3,20000,2,1,1,50000


In [89]:
# Combine one hot encoded columns with input data 
input_data=pd.concat([input_df.reset_index(drop=True),geo_encoded_df],axis=1)
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMemeber,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,20,3,20000,2,1,1,50000,1.0,0.0,0.0


In [90]:
## Encode categorical variables 
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMemeber,EstimatedSalary
0,600,France,1,20,3,20000,2,1,1,50000


In [91]:
## Concatenation of one hot encoder with geo_encoded_df 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMemeber,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,20,3,20000,2,1,1,50000,1.0,0.0,0.0


In [94]:
#Scaling 
input_scaled=scaler.fit_transform(input_df)

In [95]:
input_scaled

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [96]:
#Predict churn 
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step


array([[0.17742305]], dtype=float32)

In [97]:
prediction_proba=prediction[0][0]
print(prediction_proba)

0.17742305


In [98]:
if prediction_proba>0.5:
    print("Customer is likely to churn ")
else:
    print("Customer is not likely to churn ")

Customer is not likely to churn 
